**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

# Menu

Menu items can be made in Jupyterlab with a connection provided back to the item in ipylab.

## Main menu

We can add items to existing menus. But lets first create a new menu and add it to the main menu.

In [ ]:
import ipywidgets as ipw

import ipylab

app = ipylab.app

In [ ]:
t = app.commands.create_menu("🌈 MY CUSTOM MENU 🎌")

Menus can be added directly to the shell or nested inside other menus. Let's add this menu to main menu up top.

In [ ]:
menu = t.result()
app.main_menu.add_menu(menu)

See above that the new menu is now added

Lets populate the new menu.

In [ ]:
async def populate_menu(menu):
    await menu.add_item(command="help:about")
    await menu.add_item(type="separator")
    submenu = await menu.commands.create_menu("My submenu")
    await submenu.add_item(command="notebook:create-console")
    await menu.add_item(submenu=submenu, type="submenu")
    await menu.add_item(command="logconsole:open")

    # Open it
    menu.activate()

In [ ]:
app.to_task(populate_menu(menu))

### Built in menus

The built in menus are accessible under `app.main_menu` and can be manipulated in the same way.

In [ ]:
t = app.main_menu.file_menu.add_item(command="logconsole:open")

In [ ]:
app.main_menu.file_menu.activate()

In [ ]:
# Remove the menu item.
mc = t.result()
mc.close()

## Context menu

The app provides a global context menu. We can add items to the context menu using similar commands with an added option 'selector'.

In [ ]:
app.context_menu.add_item(submenu=menu, type="submenu")

In [ ]:
panel = ipylab.Panel([ipw.HTML("<h3>Right click to open the context menu")])

In [ ]:
panel.add_to_shell(mode=ipylab.InsertMode.split_right)

In [ ]:
menu.close()

### Selectors

Selectors are used to discriminate what items are shown in context menus which is filtered by the CSS class of the widgets.

By default Ipylab uses `app.selector` as the selector. The selector is `.ipylab-` followed by the `vpath` any dots in the vpath are replace with a hyphen '-'. the selector is added as a class for Widgets added to the shell, but with the period removed.

The scope of the context menu items can be altered by specifying the selector when adding a new item to the context menu. 

Here we define a function for notebooks by using the `.jp-Notebook` *selector*. Note a period '.' is required before the CSS class name. Multiple selectors can be be defined by adding a space between the selectors.

In [ ]:
async def show_id(ref: ipylab.ShellConnection):
    id_ = await ref.get_property("id")
    await app.dialog.show_dialog("Show id", f"Widget id is {id_}")


t = app.commands.add_command("Show id", show_id)

In [ ]:
t = app.context_menu.add_item(command=t.result(), rank=1000, selector=".jp-Notebook")

#### Limiting scope

In a similar way the scope can be narrowed, say to add a selector to a specific widget. Simply add the selector without the period '.' as a class on the widget (it needs to be a subclass of `DomWidget`).

Let's create a new `CommandRegistry` (optional), add a menu and then add the menu to the *Jupyterlab* context menu.

In [ ]:
cr = ipylab.commands.CommandRegistry(name="My command registry")
t = cr.create_menu("Extra commands")

In [ ]:
# Notice this command registry is empty
cr.all_commands

In [ ]:
# MenuConnection
mc = t.result()

In [ ]:
t = cr.add_command(
    "Open a dialog", lambda app: app.dialog.show_dialog("Custom", "This is called from a custom registry")
)

In [ ]:
cmd = t.result()
mc.add_item(command=cmd)

In [ ]:
t = app.context_menu.add_item(submenu=mc, type="submenu", selector=".WithExtraCommands")

In [ ]:
t.result()

In [ ]:
b1 = ipw.HTML(value="<h1>Context WITHOUT extra commands<h1>", layout={"border": "solid 3px blue"})
b2 = ipw.HTML("<h1>Context WITH extra commands</h1>", layout={"border": "solid 3px green"})
b2.add_class("WithExtraCommands")
panel = ipylab.Panel([b1, b2])
panel.add_to_shell()

In this way additional context can be added to specific widgets.

## On ready

**Requires per-kernel widget manager**

When the workspace is re-loaded, including when the page is refreshed, all objects in the frontend are lost. IpyWidgets get restored from the backend, but the connected objects do not. 

For this reason, creating anything that uses connections should be defined in a function and then registered with on_ready.

`on_ready` is called with the object that was registered. 

For example using `populate_menus from above:

In [ ]:
async def create_menus(app: ipylab.App):
    menu = await app.commands.create_menu("🌈 MY CUSTOM MENU 🎌")
    await app.main_menu.add_menu(menu)
    await populate_menu(menu)
    await populate_menu(app.context_menu)


app.on_ready(create_menus)

Reload the page (F5) ignoring any warnings.

The panel that was in the shell and the menus should have been restored.

Note: May require a per-kernel widget manager. See Readme for details on installation.